In [5]:
from modules import credentials
import pandas as pd
import geopandas as gpd

from sodapy import Socrata

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.calgary.ca", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.calgary.ca,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("9zvu-p8uz", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [3]:
results_df

,address,street_name,street_type,street_quad,house_number,address_type,longitude,latitude,location,point,:@computed_region_4a3i_ccfj,:@computed_region_4b54_tmc4,:@computed_region_kxmf_bzkv,:@computed_region_hq2j_w7j9,:@computed_region_p8tp_5dkv,house_alpha
0,5532 DALWOOD WY NW,DALWOOD,WY,NW,5532,Parcel,-114.16274336722893,51.114108633701804,"{'latitude': '51.114108633701804', 'longitude'...","{'type': 'Point', 'coordinates': [-114.1627433...",2,7,38,260,4,NaN
1,321 MAGNOLIA HE SE,MAGNOLIA,HE,SE,321,Parcel,-113.91484799318039,50.891335034304355,"{'latitude': '50.891335034304355', 'longitude'...","{'type': 'Point', 'coordinates': [-113.9148479...",3,4,116,258,8,NaN
2,165 CRANFIELD GR SE,CRANFIELD,GR,SE,165,Parcel,-113.98117686841111,50.88836183031347,"{'latitude': '50.88836183031347', 'longitude':...","{'type': 'Point', 'coordinates': [-113.9811768...",3,4,294,82,8,NaN
3,2 SAGE VALLEY ME NW,SAGE VALLEY,ME,NW,2,Parcel,-114.15043875308373,51.178208654570426,"{'latitude': '51.178208654570426', 'longitude'...","{'type': 'Point', 'coordinates': [-114.1504387...",2,3,265,169,10,NaN
4,714 12 ST SE,12,ST,SE,714,Parcel,-114.03128504301664,51.0419284690263,"{'latitude': '51.0419284690263', 'longitude': ...","{'type': 'Point', 'coordinates': [-114.0312850...",3,10,54,54,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5715 36 ST SE,36,ST,SE,5715,Parcel,-113.98182155645343,51.00167376409369,"{'latitude': '51.00167376409369', 'longitude':...","{'type': 'Point', 'coordinates': [-113.9818215...",3,10,159,109,11,NaN
1996,2028 URSENBACH RD NW,URSENBACH,RD,NW,2028,Parcel,-114.13641771927169,51.07025982389166,"{'latitude': '51.07025982389166', 'longitude':...","{'type': 'Point', 'coordinates': [-114.1364177...",2,13,274,61,7,NaN
1997,23 ROYAL OAK PZ NW,ROYAL OAK,PZ,NW,23,Parcel,-114.21708931166964,51.15098036814804,"{'latitude': '51.15098036814804', 'longitude':...","{'type': 'Point', 'coordinates': [-114.2170893...",2,3,252,13,9,NaN
1998,8045 48 ST SE,48,ST,SE,8045,Parcel,-113.9644035146086,50.980329594523376,"{'latitude': '50.980329594523376', 'longitude'...","{'type': 'Point', 'coordinates': [-113.9644035...",3,10,159,109,11,NaN


In [4]:
results_df.to_csv('openYYCtestdata.csv')

## Steps

1. intake address data (.csv) 24 aero dr.
2. Geocode addresses
  - Open Calgary Parcel Address
  - Google Maps
  - Foursquare
3. Time and distance matrix
  - OSRM
  - OpenRouteService
    - Both OSRM and ORS require an OpenStreetMap (OSM) Protocolbuffer (.pbf) file
      - [Geofabrik Alberta](https://download.geofabrik.de/north-america/canada/alberta.html)
        - [alberta-latest.osm.pbf](https://download.geofabrik.de/north-america/canada/alberta-latest.osm.pbf)
      - [BBBike Calgary](https://download.bbbike.org/osm/bbbike/Calgary/)
        - [Calgary.osm.pbf](https://download.bbbike.org/osm/bbbike/Calgary/Calgary.osm.pbf)
        - [Custom region](https://extract.bbbike.org/)
      - [Processed data providers](https://wiki.openstreetmap.org/wiki/Processed_data_providers)
        - interline has an API, data is updated daily [planetUtils](https://github.com/interline-io/planetutils) However it cuts Okatokes in half
      
  - Google Maps
  - Mapbox
4. Cluster into routes
  - by driving distance
  - by driving time
  - elbow rule, optimal routes
  - sklearn DBScan precalculated distance
  - test with sklearn.pairwise distance for euclidean distances
5. Time and distance matrix (the matrix from pt. 3 should be usable for this)
6. Route sequence TSP
  - OSM
  - OpenRouteService
7. Evaluate
  - R4M

### Open Source Routing Machine (OSRM)

- [Documentation](http://project-osrm.org/docs/v5.24.0/api/?language=Python)
- [GitHub](https://github.com/Project-OSRM)
- [Docker Images](https://hub.docker.com/u/osrm)

Tutorials

- [Kick Start — Open Source Routing Machine Set-Up](https://medium.com/ivymobility-developers/open-source-routing-machine-43db9ae06fb7) Docker backend
- [Python Tutorial of OSRM(Open Sourced Routing Machine) and Applications](https://www.thinkdatascience.com/post/2020-03-03-osrm/osrm/) Uses folium to display maps
- [OSRM and VROOM](https://medium.com/@calroughan/a-beginners-guide-to-installing-osrm-and-vroom-on-gcp-28fbcf1f7857)


### Open Route Service 

- [Documentation](https://openrouteservice.org/dev/#/api-docs/introduction)
- [Docker Image](https://hub.docker.com/u/openrouteservice)
- [Python SDK](https://github.com/GIScience/openrouteservice-py)

In [1]:
import openrouteservice

In [ ]:
coords = ((8.34234,48.23424),(8.34423,48.26424))

# key can be omitted for local host
client = openrouteservice.Client(base_url='http://localhost/ors')

# Only works if you didn't change the ORS endpoints manually
routes = client.directions(coords)

# If you did change the ORS endpoints for some reason
# you'll have to pass url and required parameters explicitly:
routes = client.request(
  url='/new_url',
  post_json={
      'coordinates': coords,
      'profile': 'driving-car',
      'format': 'geojson'
  })

### Google Maps

- [Python SDK](https://github.com/googlemaps/google-maps-services-python)

OSRM solves TSP not VRP
ORS solves TSP and VRP. It solves VRP using VROOM.
VROOM optimizes routes using cost tables from OSRM, ORS, or Valhala
Valhala solves TSP

Rather than using ORS or VROOM to solve the VRP. I am going to use sklearn.DBScan. After routes are clustered then I use the APIs to solve the TSP.